# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [3]:
import math, subprocess
import pandas as pd
import numpy as np
import random, math
import os
from IPython.display import display, clear_output

El ejecutable se corre indicando el archivo y un valor $p \in (0, 1)$

In [14]:
"""
    p_value
    archivo_instancia: path del archivo a evaluar
    instancia: numero de experimento
"""
def correr_experimento(p_value, archivo_instancia, instancia):
    # Leer archivo de la instancia.
    # instancia = leer_instancia(archivo_instancia
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./page_rank", archivo_instancia, str(p_value)], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    # process.stdin.write(instancia)
    # process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()
    
    # Verificar que el proceso no fallo.
    if exit_code != 0: 
        print(F"Hubo un error en la experimentacion con la instancia {archivo_instancia}")
    else:
        file_out = F"{archivo_instancia}.out"
        os.rename(file_out, F"{file_out}-{instancia}")

    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();

In [15]:
def agregar_experimentos(p_valores, exp_num, dataframe):
    # Obtenemos el experimento
    fila = dataframe.iloc[exp_num];
    for i in range(0, len(p_valores)):
        # p_valores[i] contiene el i-esimo p_valor de exp_num
        """
            fila_n["dataset"]: El dataset al que pertenece
            fila_n["n"]: La cantidad de filas y columnas
            fila_n["M"]: La cantidad de relaciones
            p_caso: El caso de p-valor a evaluar
            p_value: El p-valor
            fila_n["archivo"]: El path del archivo
        """
        experimentos.append([
            fila["dataset"], 
            fila["n"], 
            fila["M"], 
            p_valores[i], 
            fila["archivo"]
        ])
        
def agregar_experimentos_grupos(p_valores, exp_num, dataframe):
    # Obtenemos el experimento
    fila = dataframe.iloc[exp_num];
    for i in range(0, len(p_valores)):
        # p_valores[i] contiene el i-esimo p_valor de exp_num
        """
            fila_n["dataset"]: El dataset al que pertenece
            fila_n["n"]: La cantidad de filas y columnas
            fila_n["M"]: La cantidad de relaciones
            p_caso: El caso de p-valor a evaluar
            p_value: El p-valor
            fila_n["archivo"]: El path del archivo
        """
        experimentos_grupos.append([
            fila["dataset"], 
            fila["n"], 
            fila["M"], 
            p_valores[i], 
            fila["archivo"]
        ])

# Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

Se procede a hacer lo siguiente:
Para cada caso a evaluar y cada caso de p-valor hago:
- Defino un n que estará entre 0 y 100, ese n me dirá cual de los 101 casos generados se va a usar
- Para ese n, defino p_cant cantidades de p-valor y los guardo en un arreglo que será p_valores
    - Es importante tener en cuenta que ese p_cant estará sujeto al casos_p tomado
- Se pasará a agregar_experimentos el caso, el número el p_valores para que lo agregue a experimentos
- Haremos esto 10 veces por cada instancia de manera que, para caso random tomados 10 random, evaluamos cant_it p-valores para cada uno y con ello obtenemos una metrica a comparar

## Experimentación de todos menos grupos

In [4]:
investigacion={}
casos = ["ref-one-ref-lot-diversity", "ref-one-ref-lot", "referenciadas", "una-sola"]
#casos = ["referenciadas"]
experimentos = []
cant_pvalues = 200
cant_exp = 15

for caso in casos:
    investigacion[caso]=pd.read_csv(F"instancias/{caso}/indice.csv")

p_valores = []
for i in range(0, cant_pvalues):
    p_valores.append(random.uniform(0, 1))
    
for caso in casos:
    # exp_set es el set de experimentos de los 100 que puede haber en el indice.csv
    exp_set = random.sample(list(range(1, 100)), cant_exp)
    for exp in exp_set:
            #if casos_p == "random":
            #    p_valores.append(random.uniform(0, 1))
            #if casos_p == "pbajo":
            #    p_valores.append(random.uniform(0, 1))
            #if casos_p == "palto":
            #    p_valores.append(random.uniform(0, 1))       
        # En este punto ya tengo los 10 p-valores buscados, ahora solo se lo envio a agregar_experimento
        """
            Enviamos:
                p_valores: arr[] (Valores p)
                exp: int (Numero de experimento)
                casos_p: string (caso de experimento con valor p)
                investigacion[casos_c]: string (caso de experimento)
        """
        agregar_experimentos(p_valores, exp, investigacion[caso])
                    
if(not os.path.isdir("resultados")):
    os.mkdir("resultados")

In [5]:
print(len(experimentos))

12000


## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

Sabemos que nuestros resultados dependen del p-valor porque nuestras matrices son siempre las "mismas".
Es decir, dada una matriz, tenemos k p-valores con los que evaluar
Usemos eso, la cantidad de experimentos tomados es *cant_exp* y para cada uno hay *cant_pvalues*

In [6]:
filas = []

numero = 0
num_out = 1
indice = 1
exp_actual = experimentos[0][4]
# Nuestros resultados dependen del p-valor
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(num_out) + "/" + str(len(experimentos)))
    display('Dataset: ' + str(experimento[0]))
    display('Matrix Size: ' + str(experimento[1]))
    display('Relationship: ' + str(experimento[2]))
    display('P value: ' + str(experimento[3])) 
    display('Path: ' + str(experimento[4]))
    
    if(experimento[4] != exp_actual):
        numero = 0
        indice += 1
        exp_actual = experimento[4]
        
    correr_experimento(experimento[3], experimento[4], numero)
    
    filas.append([
        indice,
        experimento[0], 
        experimento[1], 
        experimento[2], 
        experimento[3], 
        experimento[4], 
        F"{experimento[4]}.out-{numero}"
    ]);
    
    num_out += 1
    numero += 1

'Experimento: 12000/12000'

'Dataset: una-sola'

'Matrix Size: 1000'

'Relationship: 36'

'P value: 0.6226501275407794'

'Path: instancias/una-sola/una-sola-93/Test-una-sola-93.txt'

In [7]:
columnas = ["n_exp", "dataset", "n", "M", "p-valor", "path", "path-out"]
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True); 

## Experimentación de referenciadas con n creciente

In [44]:
investigacion={}
casos = ["referenciadas-n"]
experimentos = []
cant_pvalues = 15

for caso in casos:
    investigacion[caso]=pd.read_csv(F"instancias/{caso}/indice.csv")

p_valores = []
for i in range(0, cant_pvalues):
    p_valores.append(random.uniform(0, 1))
    
for caso in casos:
    # exp_set es el set de experimentos de los 100 que puede haber en el indice.csv
    exp_set = list(range(1, 1000))
    for exp in exp_set:
        agregar_experimentos(p_valores, exp, investigacion[caso])
        """
            Enviamos:
                p_valores: arr[] (Valores p)
                exp: int (Numero de experimento)
                casos_p: string (caso de experimento con valor p)
                investigacion[casos_c]: string (caso de experimento)
        """
        
if(not os.path.isdir("resultados")):
    os.mkdir("resultados")

In [45]:
print(len(experimentos))

14985


## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

Sabemos que nuestros resultados dependen del p-valor porque nuestras matrices son siempre las "mismas".
Es decir, dada una matriz, tenemos k p-valores con los que evaluar
Usemos eso, la cantidad de experimentos tomados es *cant_exp* y para cada uno hay *cant_pvalues*

In [46]:
filas = []

numero = 0
num_out = 1
indice = 1
exp_actual = experimentos[0][4]
# Nuestros resultados dependen del p-valor
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(num_out) + "/" + str(len(experimentos)))
    display('Dataset: ' + str(experimento[0]))
    display('Matrix Size: ' + str(experimento[1]))
    display('Relationship: ' + str(experimento[2]))
    display('P value: ' + str(experimento[3])) 
    display('Path: ' + str(experimento[4]))
    
    if(experimento[4] != exp_actual):
        numero = 0
        indice += 1
        exp_actual = experimento[4]
        
    correr_experimento(experimento[3], experimento[4], numero)
    
    filas.append([
        indice,
        experimento[0], 
        experimento[1], 
        experimento[2], 
        experimento[3], 
        experimento[4], 
        F"{experimento[4]}.out-{numero}"
    ]);
    
    num_out += 1
    numero += 1

'Experimento: 14105/14985'

'Dataset: referenciadas-n'

'Matrix Size: 942'

'Relationship: 46961'

'P value: 0.6845442408778625'

'Path: instancias/referenciadas-n/referenciadas-n-942/Test-referenciadas-n-942.txt'

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'instancias/referenciadas-n/referenciadas-n-942/Test-referenciadas-n-942.txt.out' -> 'instancias/referenciadas-n/referenciadas-n-942/Test-referenciadas-n-942.txt.out-4'

In [ ]:
columnas = ["n_exp", "dataset", "n", "M", "p-valor", "path", "path-out"]
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado-n.csv", index=False, header=True); 

## Experimentación de Grupos

In [12]:
investigacion_grupos={}
experimentos_grupos = []
cant_pvalues = 100

investigacion_grupos["grupos"]=pd.read_csv(F"instancias/grupos/indice.csv")

# exp_set es el set de experimentos de los 100 que puede haber en el indice.csv
exp_set = list(range(0, 4))

# Instancio los 100 p-valores con los que voy a evaluar
p_valores = []
for i in range(0, cant_pvalues):
    p_valores.append(random.uniform(0, 1))
    
for exp in exp_set:
    """
        Enviamos:
            p_valores: arr[] (Valores p)
            exp: int (Numero de experimento)
            casos_p: string (caso de experimento con valor p)
            investigacion[casos_c]: string (caso de experimento)
    """
    agregar_experimentos_grupos(p_valores, exp, investigacion_grupos["grupos"])
                    
if(not os.path.isdir("resultados")):
    os.mkdir("resultados")

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

Sabemos que nuestros resultados dependen del p-valor porque nuestras matrices son siempre las "mismas".
Es decir, dada una matriz, tenemos k p-valores con los que evaluar
Usemos eso, la cantidad de experimentos tomados es *cant_exp* y para cada uno hay *cant_pvalues*

In [4]:
filas_grupos = []

numero = 0
num_out = 1
indice = 1
exp_actual = experimentos_grupos[0][4]
# Nuestros resultados dependen del p-valor


for experimento in experimentos_grupos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(num_out) + "/" + str(len(experimentos_grupos)))
    display('Dataset: ' + str(experimento[0]))
    display('Matrix Size: ' + str(experimento[1]))
    display('Relationship: ' + str(experimento[2]))
    display('P value: ' + str(experimento[3])) 
    display('Path: ' + str(experimento[4]))
    
    if(experimento[4] != exp_actual):
        numero = 0
        indice += 1
        exp_actual = experimento[4]
        
    correr_experimento(experimento[3], experimento[4], numero)
    
    filas_grupos.append([
        indice,
        experimento[0], 
        experimento[1], 
        experimento[2], 
        experimento[3], 
        experimento[4], 
        F"{experimento[4]}.out-{numero}"
    ]);
    
    num_out += 1
    numero += 1

NameError: name 'experimentos_grupos' is not defined

In [14]:
columnas = ["n_exp", "dataset", "n", "M", "p-valor", "path", "path-out"]
df_resultado = pd.DataFrame(filas_grupos, columns=columnas);
df_resultado.to_csv("resultados/resultado-grupos.csv", index=False, header=True); 

# Evaluacion de test de 15 segundos

In [17]:
investigacion={}
experimentos_segundos = []
cant_pvalues = 100
p_valores = []
for i in range(0, 50):
    p_valores.append(random.uniform(0, 1))
    
for i in range(0, len(p_valores)):
    # p_valores[i] contiene el i-esimo p_valor de exp_num
    """
        fila_n["dataset"]: El dataset al que pertenece
        fila_n["n"]: La cantidad de filas y columnas
        fila_n["M"]: La cantidad de relaciones
        p_caso: El caso de p-valor a evaluar
        p_value: El p-valor
        fila_n["archivo"]: El path del archivo
    """
    experimentos_segundos.append([
        "test-15-segundos", 
        2000, 
        12000, 
        p_valores[i], 
        F"instancias/test-15-segundos/test-15-segundos.txt"
    ])

for i in range(0, len(p_valores)):
    # p_valores[i] contiene el i-esimo p_valor de exp_num
    """
        fila_n["dataset"]: El dataset al que pertenece
        fila_n["n"]: La cantidad de filas y columnas
        fila_n["M"]: La cantidad de relaciones
        p_caso: El caso de p-valor a evaluar
        p_value: El p-valor
        fila_n["archivo"]: El path del archivo
    """
    experimentos_segundos.append([
        "test-30-segundos", 
        3000, 
        18000, 
        p_valores[i], 
        F"instancias/test-30-segundos/test-30-segundos.txt"
    ])

if(not os.path.isdir("resultados")):
    os.mkdir("resultados")

In [18]:
len(experimentos_segundos)

100

In [19]:
filas_segundos = []
numero = 0
num_out = 1
indice = 1

exp_actual = experimentos_segundos[0][4]

for experimento in experimentos_segundos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(num_out) + "/" + str(len(experimentos_segundos)))
    display('Dataset: ' + str(experimento[0]))
    display('Matrix Size: ' + str(experimento[1]))
    display('Relationship: ' + str(experimento[2]))
    display('P value: ' + str(experimento[3])) 
    display('Path: ' + str(experimento[4]))
    
    if(experimento[4] != exp_actual):
        numero = 0
        indice += 1
        exp_actual = experimento[4]
    
    correr_experimento(experimento[3], experimento[4], numero)
    
    filas_segundos.append([
        indice,
        experimento[0], 
        experimento[1], 
        experimento[2], 
        experimento[3], 
        experimento[4], 
        F"{experimento[4]}.out-{numero}"
    ]);
    
    num_out += 1
    numero += 1

'Experimento: 100/100'

'Dataset: test-30-segundos'

'Matrix Size: 3000'

'Relationship: 18000'

'P value: 0.6420735783587316'

'Path: instancias/test-30-segundos/test-30-segundos.txt'

In [20]:
columnas = ["n_exp", "dataset", "n", "M", "p-valor", "path", "path-out"]
df_resultado = pd.DataFrame(filas_segundos, columns=columnas);
df_resultado.to_csv("resultados/resultado-segundos.csv", index=False, header=True); 

## Nuevo experimento de referencias y un grupo completo

In [ ]:
investigacion={}
casos = ["referenciadas-new"]
experimentos = []
cant_pvalues = 100

for caso in casos:
    investigacion[caso]=pd.read_csv(F"instancias/{caso}/indice.csv")

p_valores = []
for i in range(0, cant_pvalues):
    p_valores.append(random.uniform(0, 1))
    
for caso in casos:
    # exp_set es el set de experimentos de los 100 que puede haber en el indice.csv
    exp_set = list(range(2,14))
    for exp in exp_set:
            #if casos_p == "random":
            #    p_valores.append(random.uniform(0, 1))
            #if casos_p == "pbajo":
            #    p_valores.append(random.uniform(0, 1))
            #if casos_p == "palto":
            #    p_valores.append(random.uniform(0, 1))       
        # En este punto ya tengo los 10 p-valores buscados, ahora solo se lo envio a agregar_experimento
        """
            Enviamos:
                p_valores: arr[] (Valores p)
                exp: int (Numero de experimento)
                casos_p: string (caso de experimento con valor p)
                investigacion[casos_c]: string (caso de experimento)
        """
        agregar_experimentos(p_valores, exp, investigacion[caso])
                    
if(not os.path.isdir("resultados")):
    os.mkdir("resultados")

In [ ]:
filas = []

numero = 0
num_out = 1
indice = 1
exp_actual = experimentos[0][4]
# Nuestros resultados dependen del p-valor
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(num_out) + "/" + str(len(experimentos)))
    display('Dataset: ' + str(experimento[0]))
    display('Matrix Size: ' + str(experimento[1]))
    display('Relationship: ' + str(experimento[2]))
    display('P value: ' + str(experimento[3])) 
    display('Path: ' + str(experimento[4]))
    
    if(experimento[4] != exp_actual):
        numero = 0
        indice += 1
        exp_actual = experimento[4]
        
    correr_experimento(experimento[3], experimento[4], numero)
    
    filas.append([
        indice,
        experimento[0], 
        experimento[1], 
        experimento[2], 
        experimento[3], 
        experimento[4], 
        F"{experimento[4]}.out-{numero}"
    ]);
    
    num_out += 1
    numero += 1

In [ ]:
columnas = ["n_exp", "dataset", "n", "M", "p-valor", "path", "path-out"]
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado-new.csv", index=False, header=True); 